# File and libraries

In [1]:
import pandas as pd


filepath=r'google_merch_store_raw_merge.csv'

sample_filepath=r'google_merch_store_ga4_merch_flattened_raw_000000000000.csv'



file_tag = "ga4_merch_store"

# DSLabs functions

In [15]:
%run "dslabs_functions.py"


# personal data functions

In [27]:
%run "data_functions.py"


data_functions lodaded


# Load

In [4]:
test_data=True
# test_data=False

if test_data==True:
    # Specify the chunk size (e.g., load 1000 lines at a time)
    chunk_size = 100000

    # Initialize an empty list to store the chunks
    chunks = []

    # Load the first chunk (or any number of chunks you want)
    for chunk in pd.read_csv(filepath, chunksize=chunk_size, parse_dates=["event_date"],date_format="%Y-%m-%d"):
        chunks.append(chunk)
        # Break after the first chunk to load only a sample
        break

    # Concatenate the chunks into a single DataFrame (optional)
    data = pd.concat(chunks)

else:
    data=pd.read_csv(filepath)


data.head()

,event_date,session_id,user_pseudo_id,event_name,event_timestamp,page_location,page_title,device_category,device_mobile_brand_name,device_mobile_model_name,...,item_id,item_name,item_brand,item_variant,item_category,price,quantity,item_revenue,item_list_index,promotion_name
0,2020-12-24,4969127937,5.157889e+06,add_payment_info,1608829882058276,https://shop.googlemerchandisestore.com/paymen...,Payment Method,desktop,Google,ChromeBook,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-12-24,972367636,6.107506e+07,scroll,1608825189361207,https://shop.googlemerchandisestore.com/asearc...,Store search results,desktop,Google,Chrome,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-12-24,7663299872,3.087717e+06,session_start,1608785340382018,https://shop.googlemerchandisestore.com/store....,Home,mobile,Huawei,<Other>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-12-24,2522966091,6.271470e+06,first_visit,1608820634895030,http://shop.googlemerchandisestore.com/google,Page Unavailable,desktop,Google,Chrome,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-12-24,8141613662,8.410505e+07,scroll,1608815456120537,https://shop.googlemerchandisestore.com/,Home,desktop,Google,Chrome,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.shape

(100000, 48)

In [6]:
summary5 = data.describe(include="all")

summary5

,event_date,session_id,user_pseudo_id,event_name,event_timestamp,page_location,page_title,device_category,device_mobile_brand_name,device_mobile_model_name,...,item_id,item_name,item_brand,item_variant,item_category,price,quantity,item_revenue,item_list_index,promotion_name
count,100000,1.000000e+05,1.000000e+05,100000,1.000000e+05,100000,99668,100000,100000,100000,...,42422,42422,42399,42143,42226,40384.000000,1803.000000,42422.000000,41104.000000,42228
unique,NaN,NaN,NaN,17,NaN,767,448,3,8,10,...,884,413,6,37,70,NaN,NaN,NaN,NaN,6
top,NaN,NaN,NaN,view_item,NaN,https://shop.googlemerchandisestore.com/,Home,desktop,Apple,Chrome,...,(not set),(not set),(not set),(not set),Home/Apparel/Men's / Unisex/,NaN,NaN,NaN,NaN,(not set)
freq,NaN,NaN,NaN,31591,NaN,10109,17312,58159,42280,27962,...,1863,2038,38931,40659,6816,NaN,NaN,NaN,NaN,39691
mean,2020-12-16 10:56:56.543999744,4.969973e+09,2.445807e+08,NaN,1.608160e+15,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,26.386366,1.861897,0.133869,6.920859,NaN
min,2020-11-01 00:00:00,6.412000e+03,1.000300e+06,NaN,1.604189e+15,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,0.000000,1.000000,NaN
25%,2020-11-23 00:00:00,2.490864e+09,5.706716e+06,NaN,1.606137e+15,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,13.000000,1.000000,0.000000,4.000000,NaN
50%,2020-12-15 00:00:00,4.941372e+09,2.062363e+07,NaN,1.608043e+15,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,22.000000,1.000000,0.000000,7.000000,NaN
75%,2021-01-09 00:00:00,7.463247e+09,5.987010e+07,NaN,1.610211e+15,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,30.000000,1.000000,0.000000,10.000000,NaN
max,2021-01-31 00:00:00,9.999433e+09,9.986757e+09,NaN,1.612138e+15,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,313.000000,204.000000,480.000000,24.000000,NaN


# pre processing

In [7]:
data['event_timestamp'] = pd.to_datetime(data['event_timestamp'], infer_datetime_format=True)

data = data.sort_values(by=['session_id', 'event_timestamp'], ascending=True)


## column drop

In [8]:
data=data.drop(['device_is_limited_ad_tracking','device_mobile_marketing_name','geo_metro','traffic_source_name'],axis=1)

# Encoding variables

## event_name

In [9]:
data['event_name'].value_counts()

event_name
view_item              31591
page_view              20827
user_engagement        15913
scroll                  7380
add_to_cart             6541
session_start           5479
first_visit             3940
select_item             3221
view_promotion          2688
begin_checkout          1180
view_search_results      401
add_shipping_info        260
purchase                 228
add_payment_info         193
select_promotion         129
click                     22
view_item_list             7
Name: count, dtype: int64

In [29]:
calculate_returning_user_percentage_by_group(data, 'event_name')


,event_name,total_events,returning_users,returning_user_percentage
4,click,22,15,68.181818
0,add_payment_info,193,116,60.103627
7,purchase,228,136,59.649123
3,begin_checkout,1180,679,57.542373
14,view_item_list,7,4,57.142857
1,add_shipping_info,260,130,50.000000
2,add_to_cart,6541,2997,45.818682
9,select_item,3221,1405,43.619994
13,view_item,31591,13032,41.252255
8,scroll,7380,2581,34.972900


In [10]:
event_name_enc=encode_and_sort_alphabetically_values(data,'event_name')


{'add_payment_info': 1, 'add_shipping_info': 2, 'add_to_cart': 3, 'begin_checkout': 4, 'click': 5, 'first_visit': 6, 'page_view': 7, 'purchase': 8, 'scroll': 9, 'select_item': 10, 'select_promotion': 11, 'session_start': 12, 'user_engagement': 13, 'view_item': 14, 'view_item_list': 15, 'view_promotion': 16, 'view_search_results': 17}


## device_name

In [26]:
calculate_sessions_per_user_ratio(data, 'device_category')


,device_category,total_events,returning_users,returning_user_percentage
1,mobile,39649,14159,35.710863
0,desktop,58159,20212,34.753005
2,tablet,2192,716,32.664234


In [11]:
device_category_enc=encode_and_sort_by_value_counts(data,'device_category')

{'tablet': 1, 'mobile': 2, 'desktop': 3}


# encoding job

In [12]:


encoding: dict[str, dict[str, int]] = {
    "event_name": event_name_enc,
    "device_category": device_category_enc,

}


data_encoded = data.replace(encoding, inplace=False)
data_encoded.head()

,event_date,session_id,user_pseudo_id,event_name,event_timestamp,page_location,page_title,device_category,device_mobile_brand_name,device_mobile_model_name,...,item_id,item_name,item_brand,item_variant,item_category,price,quantity,item_revenue,item_list_index,promotion_name
68550,2020-12-15,6412,2.407095e+07,9,1970-01-19 14:40:56.034582980,https://shop.googlemerchandisestore.com/myacco...,The Google Merchandise Store - My Account,1,Xiaomi,<Other>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41070,2020-12-15,6412,2.407095e+07,14,1970-01-19 14:40:56.112799509,https://shop.googlemerchandisestore.com/store....,Home,1,Xiaomi,<Other>,...,GGOEGXXX1427,Google Austin Campus Ladies Tee,(not set),(not set),Home/Campus Collection/,25.0,NaN,0.0,6.0,(not set)
71610,2020-12-12,24095,5.089247e+07,4,1970-01-19 14:35:53.470139496,https://shop.googlemerchandisestore.com/yourin...,Checkout Your Information,2,Apple,iPhone,...,9196652,Google Heathered Pom Beanie,Google,Single Option Only,New,12.0,1.0,0.0,NaN,(not set)
76245,2020-12-12,24095,5.089247e+07,8,1970-01-19 14:35:53.618319786,https://shop.googlemerchandisestore.com/orderc...,Checkout Confirmation,2,Apple,iPhone,...,9196676,Google Speckled Beanie Grey,Google,Single Option Only,New,16.0,1.0,16.0,NaN,NaN
13103,2020-12-18,662660,3.228634e+07,13,1970-01-19 14:45:19.813224433,https://shop.googlemerchandisestore.com/Google...,Apparel | Google Merchandise Store,3,Apple,Safari,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
